In [84]:
#nuclio: ignore
import nuclio

In [85]:
!pip install nuclio_sdk

In [86]:
%%nuclio cmd -c
pip install opencv-python-headless
pip install pandas
pip install v3io_frames
pip install nuclio_sdk

In [87]:
import nuclio_sdk
import json
import os
import v3io_frames as v3f
from requests import post
import base64
import numpy as np
import pandas as pd
import cv2
import random
import string
from datetime import datetime
from os import getenv, path

In [88]:
%%nuclio env
DATA_PATH = /User/faces/dataset/
V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}

%nuclio: setting 'DATA_PATH' environment variable
%nuclio: setting 'V3IO_ACCESS_KEY' environment variable


In [89]:
is_partitioned = True #os.environ['IS_PARTITIONED']

def generate_file_name(current_time, is_partitioned):
    filename_str = current_time + '.jpg'
    if is_partitioned == "true":
        filename_str = current_time[:-4] + "/" + filename_str
    return filename_str

def generate_image_path(filename, is_unknown):
    file_name = filename
    if is_unknown:
        pathTuple = (os.environ['DATA_PATH'] + 'label_pending', file_name) 
    else:
        pathTuple = (os.environ['DATA_PATH'] + 'images', file_name)   
    path = "/".join(pathTuple)
    return path

def jpg_str_to_frame(encoded):
    jpg_original = base64.b64decode(encoded)
    jpg_as_np = np.frombuffer(jpg_original, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def save_image(encoded_img, path):
    frame = jpg_str_to_frame(encoded_img)
    directory = '/'.join(path.split('/')[:-1])
    if not os.path.exists(directory):
        os.mkdir(directory)
    cv2.imwrite(path, frame)
    
def write_to_kv(client, face, path, camera, time):
    USER_NAME = getenv('V3IO_USERNAME')
    ENCODINGS_PATH = '/'.join([USER_NAME,'faces','encodings']) 
    rnd_tag = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
    name = face['name']
    label = face['label']
    encoding = face['encoding']
    
    new_row = {}  
    new_row = {'c' + str(i).zfill(3): encoding[i] for i in range(128)}
    if name != 'unknown': 
        new_row['label'] = label
        new_row['fileName'] = name.replace(' ', '_') + '_' + rnd_tag
    else:
        new_row['label'] = -1
        new_row['fileName'] = 'unknown_' + rnd_tag
            
    new_row['imgUrl'] = path
    new_row['camera'] = camera
    new_row['time'] = datetime.strptime(time, '%Y%m%d%H%M%S')
    new_row_df = pd.DataFrame(new_row, index=[0])
    new_row_df = new_row_df.set_index('fileName')
    print(new_row['fileName'])
    client.write(backend='kv', table=ENCODINGS_PATH, dfs=new_row_df) #, save_mode='createNewItemsOnly')   
    
def init_context(context):
    setattr(context.user_data, 'client', v3f.Client("framesd:8081", container="users"))
    
def handler(context, event):
        context.logger.info('extracting metadata')
        body = json.loads(event.body)
        time = body['time']
        camera = body['camera']
        encoded_img = body['content']
        
        content = {'img': encoded_img}

        context.logger.info('calling model server')
        resp = context.platform.call_function('default-recognize-faces', event)
        faces = json.loads(resp.body)
        
        context.logger.info('going through discovered faces')
        for face in faces:
            is_unknown = face['name'] == 'unknown'
            file_name = generate_file_name(time, is_partitioned) 
            path = generate_image_path(file_name, is_unknown)
            
            context.logger.info('saving image to file system')
            save_image(encoded_img, path)
            
            context.logger.info('writing data to kv')
            write_to_kv(context.user_data.client, face, path, camera, time)
            
        return faces

In [90]:
#nuclio: end-code

In [91]:
# converts the notebook code to deployable function with configurations
from mlrun import code_to_function, mount_v3io
fn = code_to_function('video-api-server', kind='nuclio')

# set the API/trigger, attach the home dir to the function
fn.with_http(workers=2).apply(mount_v3io())

# set environment variables
fn.set_env('DATA_PATH' ,'/User/faces/dataset/')
fn.set_env('V3IO_ACCESS_KEY', os.environ['V3IO_ACCESS_KEY'])
addr = fn.deploy(project='default')

> 2020-10-25 10:43:34,745 [info] deploy started
[nuclio] 2020-10-25 10:43:36,855 (info) Build complete
[nuclio] 2020-10-25 10:43:41,924 done updating default-video-api-server, function address: 192.168.224.50:31240
> 2020-10-25 10:43:41,933 [warning] warning!, server (0.5.1) and client (0.5.2) ver dont match
